In [17]:
#loading libraries
import pandas as pd
import numpy as np
import scipy as sp
import plotly.offline as py

from sklearn.model_selection import GridSearchCV

import plotly.graph_objs as go
from sklearn import linear_model, cross_validation

import datetime

In [18]:
#loading data
df_X_train = pd.read_csv('X_train.csv')
n_train = df_X_train.shape[0]

df_X_test = pd.read_csv('X_test.csv')
n_test = df_X_test.shape[0]

df_y_train = pd.read_csv('challenge_output_data_training_file_predict_air_quality_at_the_street_level.csv')
#y_train is an array without the 'ID' column
y_train = df_y_train['TARGET']


In [19]:
#quick look over the data
pd.set_option('display.max_columns', 60)
#df_X_train.info()
df_X_train.head()
#Remark : convert boolean to 1 0 numeric
#Remark 2 : pollutant is of type Object

,ID,hlres_50,green_5000,hldres_50,daytime,route_100,precipintensity,precipprobability,hlres_1000,temperature,is_calmday,route_1000,roadinvdist,port_5000,windbearingsin,cloudcover,hldres_100,natural_5000,hlres_300,hldres_300,route_300,station_id,pressure,route_500,hlres_500,hlres_100,pollutant,industry_1000,zone_id,windbearingcos,windspeed,hldres_500,hldres_1000
0,0,NaN,5172542.5,3755.19043,72.0,NaN,0.6096,0.61,NaN,9.49,False,8027.166504,0.04678,NaN,-0.587785,1.00,13612.243164,5172542.5,NaN,114993.5625,NaN,16.0,1029.349976,NaN,NaN,NaN,NO2,NaN,0.0,0.809017,6.55,357436.1875,1542650.0
1,1,NaN,5172542.5,3755.19043,72.0,NaN,0.6096,0.61,NaN,9.49,False,8027.166504,0.04678,NaN,-0.587785,1.00,13612.243164,5172542.5,NaN,114993.5625,NaN,16.0,1029.349976,NaN,NaN,NaN,PM10,NaN,0.0,0.809017,6.55,357436.1875,1542650.0
2,2,NaN,5172542.5,3755.19043,73.0,NaN,0.0965,0.14,NaN,8.22,False,8027.166504,0.04678,NaN,-0.069756,1.00,13612.243164,5172542.5,NaN,114993.5625,NaN,16.0,1029.560059,NaN,NaN,NaN,NO2,NaN,0.0,0.997564,4.47,357436.1875,1542650.0
3,3,NaN,5172542.5,3755.19043,73.0,NaN,0.0965,0.14,NaN,8.22,False,8027.166504,0.04678,NaN,-0.069756,1.00,13612.243164,5172542.5,NaN,114993.5625,NaN,16.0,1029.560059,NaN,NaN,NaN,PM10,NaN,0.0,0.997564,4.47,357436.1875,1542650.0
4,4,NaN,5172542.5,3755.19043,74.0,NaN,0.0000,0.00,NaN,7.58,False,8027.166504,0.04678,NaN,-0.104528,0.97,13612.243164,5172542.5,NaN,114993.5625,NaN,16.0,1029.660034,NaN,NaN,NaN,NO2,NaN,0.0,0.994522,4.11,357436.1875,1542650.0


In [20]:
#First concatenate train and test variable
df_X_all = pd.concat([df_X_train, df_X_test], axis = 0)

#convert boolean feature iscalm
df_X_all.is_calmday = df_X_all.is_calmday.astype(int)
df_X_train.is_calmday = df_X_train.is_calmday.astype(int)
df_X_test.is_calmday = df_X_test.is_calmday.astype(int)

#list of pollutants to perform clustering
pollutant_names = ['NO2', 'PM10', 'PM2_5']

First model, we take into account features without NA (i.e. all weather related measures,
and non categorical data (i.e no zone_id nor station_id)

In [21]:
#list of features related to weather
features = ['daytime', 'precipintensity', 'precipprobability', 'temperature', 
            'is_calmday', 'windbearingsin', 'cloudcover', 'pressure',
            'windbearingcos', 'windspeed']

X_numeric = df_X_all[features]
#Splitting train and test tables
X_train = df_X_train[features]
X_test = df_X_test[features]

#verifying dimensions
[X_test.shape[0] + X_train.shape[0], df_X_all.shape[0]]


[749060, 749060]

In [22]:
#Clustering by pollutants 
X_train_pollutants = [X_train.ix[df_X_train['pollutant'] == name, : ] for name in pollutant_names]
X_test_pollutants = [X_test.ix[df_X_test['pollutant'] == name, : ] for name in pollutant_names]
y_train_pollutants = [y_train.ix[df_X_train['pollutant'] == name] for name in pollutant_names]

In [23]:
index_pollutants_test = df_X_test['pollutant']
alpha_grid = np.linspace(0.1,10,5)
clf_pred = [] #list to store the results
clf_scores = [] #list to store the results
clf =GridSearchCV(linear_model.Ridge(), param_grid=dict(alpha=alpha_grid),
                   scoring='neg_mean_squared_error', n_jobs = 3)
for k in range(3):

    clf.fit(X_train_pollutants[k], y_train_pollutants[k])
    clf_scores.append(clf.score(X_train_pollutants[k], y_train_pollutants[k]))
    clf_pred.append(clf.predict(X_test))

In [24]:
y_pred = 0
for name, k in zip(pollutant_names, range(3)):
    y_pred += clf_pred[k] * (index_pollutants_test == name)
y_pred.shape
y_pred.to_csv('y_pred.csv')

Conclusion : For all the values of lambda the score is the same around -77 FOR THE LAST CLUSTER (PM_25)
We tried a Lasso regression, but it was worse. Best estimator for Lasso is for lambda almost zero. with same score as the Ridge regression. However the score decreases as soon as lambda inscreases (for the Lasso)

In [26]:
clf_scores

[-294.88848325212041, -164.63182940205482, -76.066759624695464]

Conclusion2 : looking at the scores of the 3 clusters: [-294.88848622247036, -164.63182940111329, -76.066774579253533]
The regression performs better on the last pollutant PM2_5. 
It can be explain by the fact that it represents onlly 7% of the data. The other pollutants
represents around 45 % that's why they could be much harder to be predicted

In [28]:
print(X_test_pollutants[0].shape[0]/n_test,
      X_test_pollutants[1].shape[0]/n_test,
      X_test_pollutants[2].shape[0]/n_test)


0.44877380845555365 0.4783559494966616 0.07287024204778475


Idea : remove the is_calm day variable which is the only boolean in the model. Conclusion : the score of the model got worse : [-302.30341071144005, -165.57511041231322, -76.279604179684753]

Seconde Step : adding a sub cluster zone_id for each pollutant


In [29]:
index_pollutants_train = df_X_train['pollutant'] == 'NO2'


In [33]:
#Clustering by pollutants and zone
zone_name = df_X_train.zone_id.unique()
X_train_pollutants_zones = [X_train.ix[(df_X_train['pollutant'] == name) & (df_X_train['zone_id'] == zone),: ] for name in pollutant_names for zone in zone_name]
X_test_pollutants_zones = [X_test.ix[(df_X_test['pollutant'] == name) & (df_X_test['zone_id'] == zone),: ] for name in pollutant_names for zone in zone_name]
y_train_pollutants_zones = [y_train.ix[(df_X_train['pollutant'] == name) & (df_X_train['zone_id'] == zone)] for name in pollutant_names for zone in zone_name]


In [ ]:
#WARNING : we got empty list for pollutant PM_25 in zone 0, 3, 4, 5
#So we want to skip those index, to not get errors while fitting the model
index_pollutants_test = df_X_test['pollutant']
index_zone_test = df_X_test['zone_id']
alpha_grid = np.linspace(0.1,10,4)
clf_pred = [] #list to store the results
clf_scores = [] #list to store the results
clf =GridSearchCV(linear_model.Ridge(), param_grid=dict(alpha=alpha_grid),
                   scoring='neg_mean_squared_error', n_jobs = 2)
for k in range(18):
    #skipping indexes with no values
    if (k == 15 or k == 16 or k ==12 or k ==17):
        clf_scores.append(0)
        clf_pred.append(0)
    else:
        clf.fit(X_train_pollutants_zones[k], y_train_pollutants_zones[k])
        clf_scores.append(clf.score(X_train_pollutants_zones[k], y_train_pollutants_zones[k]))
        clf_pred.append(clf.predict(X_test))

In [35]:
clf_scores

[-188.10797150456611,
 -372.28797532507468,
 -289.32022973327344,
 -434.66643646269955,
 -125.86472367315132,
 -132.43254610525133,
 -134.87891075915979,
 -129.61229571170281,
 -128.48226417505771,
 -155.02130475833309,
 -112.4174475659284,
 -252.44855124184241,
 0,
 -82.752149603245769,
 -69.251062166140784,
 0,
 0,
 0]

In [210]:
y_pred = 0
y_pred += clf_pred[0] * ((index_pollutants_test == 'NO2') & (index_zone_test == 0.0))
y_pred += clf_pred[1] * ((index_pollutants_test == 'N02') & (index_zone_test == 1.0))
y_pred += clf_pred[2] * ((index_pollutants_test == 'NO2') & (index_zone_test == 2.0))
y_pred += clf_pred[3] * ((index_pollutants_test == 'NO2') & (index_zone_test == 3.0))
y_pred += clf_pred[4] * ((index_pollutants_test == 'NO2') & (index_zone_test == 4.0))
y_pred += clf_pred[5] * ((index_pollutants_test == 'NO2') & (index_zone_test == 5.0))
y_pred += clf_pred[6] * ((index_pollutants_test == 'PM10') & (index_zone_test == 0.0))
y_pred += clf_pred[7] * ((index_pollutants_test == 'PM10') & (index_zone_test == 1.0))
y_pred += clf_pred[8] * ((index_pollutants_test == 'PM10') & (index_zone_test == 2.0))
y_pred += clf_pred[9] * ((index_pollutants_test == 'PM10') & (index_zone_test == 3.0))
y_pred += clf_pred[10] * ((index_pollutants_test == 'PM10') & (index_zone_test == 4.0))
y_pred += clf_pred[11] * ((index_pollutants_test == 'PM10') & (index_zone_test == 5.0))
y_pred += clf_pred[12] * ((index_pollutants_test == 'PM2_5') & (index_zone_test == 0.0))
y_pred += clf_pred[13] * ((index_pollutants_test == 'PM2_5') & (index_zone_test == 1.0))
y_pred += clf_pred[14] * ((index_pollutants_test == 'PM2_5') & (index_zone_test == 2.0))
y_pred += clf_pred[15] * ((index_pollutants_test == 'PM2_5') & (index_zone_test == 3.0))
y_pred += clf_pred[16] * ((index_pollutants_test == 'PM2_5') & (index_zone_test == 4.0))
y_pred += clf_pred[17] * ((index_pollutants_test == 'PM2_5') & (index_zone_test == 5.0))

y_pred.shape
y_pred.to_csv('y_pred3.csv')

The resulting loss is of 336. So it's better not to separate each group of pollutant and zone id
but to perform a ridge regression on the whole data